# Utilizing advanced features - Regex <ins>argpacks</ins>
In this tutorial we will be exploring how to use advanced features of the framework. We will be using common terminology found in the repo's README.md - refer to that for any underlined terms that need clarification. Additionally, we will be building upon the material covered in the [Basic Test Config](./BasicTestConfig.ipynb); please review that tutorial if you haven't already. Anything in `"code-quoted"` format refers specifically to the test config, and anything in <ins>underlined</ins> format refers to specific terminology that can be found in the [README.md](../README.md).


In [ ]:
# Get notebook location
shellReturn = !pwd
notebookDirectory = shellReturn[0]
print( "Working from " + notebookDirectory )

Advanced usage of the json config `"<regex>::<argpack>"` option under `"arguments"` will be the focus of this tutorial :


In [ ]:
# Output template file documenting options
from IPython.display import Markdown as md
md( "```jsonc\n" + open( notebookDirectory + "/../.ci/template.json", "r" )
                        .read()
                        .split( "// dict of argpacks" )[1]
                        .split( "// NO MORE KEYWORDS AT THIS LEVEL" )[0] + 
   "\n```" )

## Regex-based <ins>argpacks</ins>

The following section assumes you have a general understanding of regular expressions. For further reading please refer to :
* [the wikipedia article](https://en.wikipedia.org/wiki/Regular_expression#Basic_concepts)
* an excellent [open-source guide available in many languages](https://github.com/ziishaned/learn-regex)
* a python-specific [`re` introduction](https://docs.python.org/3/howto/regex.html#regex-howto)
* (something I personally find incredibly useful) an [online regex tester with in-depth explanations](https://regex101.com/)

When using the `"<regex>::"`-style <ins>argpack</ins>, it is important to note a few things : 
1. The regex flavor used is python's `re` module
2. The python if-check uses `re.match()` with no flags ([`re` flags](https://docs.python.org/3/library/re.html#flags))
3. __ALL__ inherited arguments are attempted to be applied only when steps begin execution
4. The regex is applied to the full <ins>ancestry</ins>
5. To override the regex-<ins>argpack</ins>, it must match wholly `"<regex>::<argpack>"`
6. The `"<argpack>"` portion of `"<regex>::<argpack>"` is what is used to sort order


For (1) and (2), please refer to the `re` reference links. Simply put, (1) gives wide flexibility on the types of regex constructs that may be used and (2) means the regex is interpreted as literal as possible, i.e. case-sensitive, `^` and `$` only apply to beginning and end of string, and `.` does not match newline.


The importance of (3) is that the cummulative set of arguments inherited in a <ins>step</ins>'s <ins>ancestry</ins> is always applied. When not using regex-<ins>argpack</ins>, one should be careful to only pass things down to appropriate steps. Thus, with the advantages of regex-based conditional application of <ins>argpacks</ins> it is possible to become lazy or sloppy in heavy reliance of this feature. This could be thought of akin to global variables in that solely relying on regexes to route arguments from the top-level `"arguments"` option pollutes the <ins>argpack</ins> namespace unnecessarily. Writing regexes is already complex enough, and having _all_ the regexes applied to _all_ <ins>steps</ins> but only needing them to apply to a few under a single specific <ins>test</ins> may be a disaster waiting to happen and easier solved by scoping the regex to just the <ins>test</ins>'s `"argument"` option.

Point (4) provides corollary to (3): because the regex is applied to the <ins>ancestry</ins>, to make maximal use of this greater flexibility in application of the arguments one should be mindful of test and step naming in conjunction with writing well-defined regexes. Keeping this in mind for a <ins>test config</ins>, one could devise specific naming conventions allowing for specific test or step filtering. 

For instance, if I have a series of steps across multiple tests but all require a specific set of arguments when executing the "build" step of each test, I can extract the common build arguments and place them under a `".*build.*::build_args"` <ins>argpack</ins> if my "build" steps are prefixed with `"build"`. Furthermore, if my build steps across tests slightly differ based on compiler I can name my steps `"build-gcc"`, `"build-icc"`, `"build-clang"` and so on writing other <ins>argpacks</ins> such as `".*-gcc.*::gcc_env"` to load a specific environment.


Finally, (5) and (6) are restatements from the [README.md](../README.md) "How it works"-><ins>Submit Options</ins> subsection. These matter when argument order or overriding an <ins>argpack</ins> is necessary. Generally speaking, if you find you need to often override an <ins>argpack</ins>, whether regex or not (but especially if regex) you are most likely overcomplicating things. Consider rescoping the arguments or moving them to specific `"arguments"` option inside a <ins>step</ins> (not the `"submit_options"`)

### Simple regex-<ins>argpack</ins> config

Let's start with something small : a test with two types of steps - a sender and receiver - with prefixes `send` and `recv`. We'd like for each to identify themselves with a defined string before output.


In [ ]:
%%bash -s "$notebookDirectory"
cat << EOF > $1/../our-config.json
{
  "regex-test" : 
  {
    "submit_options" :
    {
      "arguments" :
      {
        ".*send.*::send_prefix" : [ "[send] " ],
        ".*recv.*::recv_prefix" : [ "[recv] " ]
      }
    },
    "steps" :
    {
      "send-step0" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "Hello!" ] },
      "recv-step1" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "Hello back!" ] },
      "send-step2" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "Ping 1" ] },
      "send-step3" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "Ping 2" ] },
      "recv-step4" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "Pings received" ] }
    }
  }
}
EOF

echo "$( realpath $1/../our-config.json ) :"
cat $1/../our-config.json

Let's run this config with `--forceSingle` to our <ins>run script</ins> to see the output easier. Likewise, we will use `--inlineLocal` to avoid the need to peek at log files.

In [ ]:
%%bash -s "$notebookDirectory"
$1/../.ci/runner.py $1/../our-config.json -t regex-test --forceSingle --inlineLocal

When inspecting the output, we do see in the lines following `Submitting step <stepname>...` where <ins>argpacks</ins> are applied the appropriate prefix is selected, but our prefix is being applied as a suffix. Even more explicit, the line after `Running command:` verbatim outputs the command and arguments showing two argument strings being supplied with our "prefix" last. Not quite what we wanted...

<div class="alert alert-block alert-info">
<b>Recall:</b>
The order of <ins>arguments</ins> applied is step-specifics first, then all cummulative <ins>argpacks</ins> from <code>"submit_options"</code> in alphabetical order with conflicts resolved by order of first appearance.
</div>


When applied to a scipt in a manner that is expected, the regex-<ins>argpacks</ins> can become a very powerful feature. Let's further explore how one might apply this to a broader scope of procedures.

### Advanced regex-<ins>argpack</ins> config

To get a feel for how best the regex feature of <ins>argpacks</ins> may be used, we will "build" a more complex set of <ins>steps</ins> that wouldn't just echo out the arguments. The term "build" here should be taken figuratively, as actually developing the logic to be outlined is beyond the scope of this tutorial. We will still be using the `echo_normal.sh` script for ease of use.

First, assume we have a build system (CMake, Make, etc.) who can alternate build types based on flags or configuration inputs. Assuming builds can be run in parallel, we _could_ have each build test run as separate <ins>tests</ins> unto themselves. However, these could also be categorically one <ins>test</ins> -  a compilation test. Let's do just that with the following assumptions:
* `build.sh` can facilitate our flags into the corresponding build system
* `-o` determines the build output location
* `-d` sets debug mode
* `--mpi` sets build with MPI
* `--omp` sets build with OpenMP
* `--double` sets build with double precision
* `-a` enables feature A
* `-b` enables feature B
* `-c` enables feature C which is mutually exclusive with A

We don't want a combination of every option, just a select few for our most critical tests. 


In [ ]:
%%bash -s "$notebookDirectory"
cat << EOF > $1/../our-config.json
{
  "regex-test" : 
  {
    "submit_options" :
    {
      "arguments" :
      {
        ".*dbg.*::dbg_flag" : [ "-d" ],
        ".*mpi.*::mpi_flag" : [ "--mpi" ],
        ".*omp.*::omp_flag" : [ "--omp" ],
        ".*fp64.*::fp64_flag" : [ "--double" ],
        ".*ft[^A]*A.*::feature_a" : [ "-a" ],
        ".*ft[^B]*B.*::feature_b" : [ "-b" ],
        ".*ft[^C]*C.*::feature_b" : [ "-c" ]
      }
    },
    "steps" :
    {
      "build-omp-fp32-dbg"  : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-omp-fp32-dbg" ] },
      "build-omp-fp32-ftA"  : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-omp-fp32-ftA" ] },
      "build-omp-fp32-ftAB" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-omp-fp32-ftAB" ] },
      "build-omp-fp32-ftBC" : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-omp-fp32-ftBC" ] },
      "build-omp-fp64-ftB"  : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-omp-fp64-ftB" ] },
      "build-mpi-fp32-dbg"  : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-mpi-fp32-dbg" ] },
      "build-mpi-fp32"      : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-mpi-fp32" ] },
      "build-mpi-fp64"      : { "command" : "./tests/scripts/echo_normal.sh", "arguments" : [ "-o", "build-mpi-fp64" ] }
    }
  }
}
EOF

echo "$( realpath $1/../our-config.json ) :"
cat $1/../our-config.json

Let's run this code and see how flags are routed to the appropriate tests.

In [ ]:
%%bash -s "$notebookDirectory"
$1/../.ci/runner.py $1/../our-config.json -t regex-test --forceSingle --inlineLocal

By solely controlling the apt naming of our steps we can get the correct flags applied to the respective step. This, of course, is a heavily simplified example.